In [12]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from salishsea_tools import evaltools as et, viz_tools
import os
import scipy.interpolate as sinterp
import cmocean
import netCDF4 as nc
import xarray as xr

In [13]:
path_run = PATH= '/home/jvalenti/scratch/run_SHEM/SHEM18/' 
SalishSeadf = pd.read_csv('/home/jvalenti/MOAD/analysis-jose/notebooks/SHEM/SalishSea_MesozoopModelvsObsMatches_2015to2019_Eval202111.csv', index_col=0,header=0,parse_dates= ['Date','dtUTC'])

In [3]:
# date_original = SalishSeadf['dtUTC']
# date_altered = []

# for t in date_original:
#     date_altered.append(t.replace(year=2021))


# SalishSeadf['dtUTC'] = date_altered


In [14]:
def make_filename(path_run,start,day=0, var='biol_T', res='h'):
    """Construct path prefix for local SHEM results given date object and paths dict
    """
    date  =  (start + timedelta(days=day))
    folder = date.strftime("%d%b%y").lower()
    prefix = os.path.join(path_run, f'{folder}/')
    fname = []
    try:
        for file in os.listdir(prefix):
            if (var in file) and ('_1'+res) in file:
                fname.append(file)
        if len(fname)>1:
            print('more than one file found') 
        try:
            path = os.path.join(prefix, fname[0])
            return path
        except IndexError:
            print('no file available')
    except FileNotFoundError:
        print('no file available')
    

In [15]:
M_path = '/home/jvalenti/MOAD/'
mask = xr.open_dataset(M_path+'grid2/mesh_mask202108_TDV.nc')

In [19]:
# example = SalishSeadf.iloc[0]
# start = example.dtUTC.to_pydatetime()    
# mmolNm2 = (xr.open_dataset(make_filename(path_run,start)).mesozooplankton[0,example.k_upper:example.k_lower+1,example.j,example.i]
#            *mask.e3t_0[0,example.k_upper:example.k_lower+1,example.j,example.i]).sum()

In [18]:
total_mine = np.zeros(len(SalishSeadf))
for n in range(len(SalishSeadf)):
    example = SalishSeadf.iloc[n]
    start = example.dtUTC.to_pydatetime()    
    jj = example.j
    ii = example.i
    k_u = example.k_upper
    k_l = example.k_lower
    e3tt = np.array(mask.e3t_0[0,k_u:k_l+1,jj,ii])
    masktt = np.array(mask.tmask[0,k_u:k_l+1,jj,ii])
    res = 'd'
    var = 'biol_T'
    try:
        total_mine[n] = sum(np.array(xr.open_dataset(make_filename(path_run,start,var=var,res=res)).mesozooplankton[0,k_u:k_l+1,jj,ii])*e3tt*masktt)/sum(e3tt*masktt)
    except ValueError or FileNotFoundError:
        total_mine[n]=np.nan

no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file available
no file av

In [20]:
SalishSeadf['SHEM_mesozooplankton']=total_mine

In [21]:
SalishSeadf.to_csv('evaldata_shem2.csv')

In [22]:
SalishSeadf['SHEM_mesozooplanktonDI']=(SalishSeadf['SHEM_mesozooplankton'])*SalishSeadf['Z_lower']

In [23]:
# define log transform function with slight shift to accommodate zero values
def logt(x):
  return np.log10(x+.001)
# define inverse log transform with same shift
def logt_inv(y):
    return 10**y-.001

In [24]:
#Convert SHEMel values to mg C m-3 by muliplying value * C:N of 5.7 * molecular weight of C
SalishSeadf['L10SHEM_mesozooplankton']=logt(SalishSeadf['SHEM_mesozooplankton']*5.7*12)
SalishSeadf['L10SHEM_mesozooplanktonDI']=logt(SalishSeadf['SHEM_mesozooplanktonDI']*5.7*12/1000)
SalishSeadf['logt_invL10SHEM_mesozooplanktonDI']=logt_inv(SalishSeadf['L10SHEM_mesozooplanktonDI'])

In [25]:
print(monthlymean['Total'].max())
print(monthlymean['mod_mesozooplankton'].max()*5.7*12)

NameError: name 'monthlymean' is not defined